**Recuperar noticias LaJornada**

In [3]:
#V2 fecha normalizada

import requests
import xml.etree.ElementTree as ET
import csv
from datetime import datetime

urls_rss = [
    "https://www.jornada.com.mx/rss/deportes.xml?v=1",
    "https://www.jornada.com.mx/rss/economia.xml?v=1",
    "https://www.jornada.com.mx/rss/ciencias.xml?v=1",
    "https://www.jornada.com.mx/rss/cultura.xml?v=1"
]

secciones_nombres = {
    "https://www.jornada.com.mx/rss/deportes.xml?v=1": "Deportes",
    "https://www.jornada.com.mx/rss/economia.xml?v=1": "Economía",
    "https://www.jornada.com.mx/rss/ciencias.xml?v=1": "Ciencias",
    "https://www.jornada.com.mx/rss/cultura.xml?v=1": "Cultura"
}

nombre_archivo_csv = 'LaJornada.csv'
encabezados = ['Fuente', 'Titulo', 'Resumen de Contenido', 'Seccion', 'URL', 'Fecha']

with open(nombre_archivo_csv, mode='w', newline='', encoding='utf-8') as archivo:
    escritor = csv.writer(archivo)
    escritor.writerow(encabezados)
    for url in urls_rss:
        respuesta = requests.get(url)
        if respuesta.status_code == 200:
            contenido = respuesta.content.decode('utf-8')
            raiz = ET.fromstring(contenido)
            for elemento in raiz.findall('.//item'):
                titulo = elemento.find('title').text
                enlace = elemento.find('link').text
                descripcion = elemento.find('description').text
                fecha_str = elemento.find('pubDate').text
                fecha = datetime.strptime(fecha_str, '%a, %d %b %Y %H:%M:%S %Z')
                fecha_formateada = fecha.strftime('%d/%m/%Y')
                seccion = secciones_nombres[url]
                escritor.writerow(['La Jornada', titulo, descripcion, seccion, enlace, fecha_formateada])
print('Datos guardados en el archivo CSV.')


Datos guardados en el archivo CSV.


**Recuperar noticias Expansión**

In [4]:
#V2 Fecha normalizada
import requests
import xml.etree.ElementTree as ET
import csv
from datetime import datetime

urls_rss = [
    "https://expansion.mx/rss/economia.xml",
    "https://expansion.mx/rss/tecnologia",
]

nombre_archivo_csv = 'Expansion.csv'

encabezados = ['Fuente', 'Titulo', 'Resumen de Contenido', 'Seccion', 'URL', 'Fecha']

encabezados_peticion = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

espacios_nombres = {
    'dc': 'http://purl.org/dc/elements/1.1/',
    'content': 'http://purl.org/rss/1.0/modules/content/',
    'media': 'http://search.yahoo.com/mrss/'
}

with open(nombre_archivo_csv, mode='w', newline='', encoding='utf-8') as archivo:
    escritor = csv.writer(archivo)
    escritor.writerow(encabezados)
    for url in urls_rss:
        respuesta = requests.get(url, headers=encabezados_peticion)
        if respuesta.status_code == 200:
            contenido = respuesta.content.decode('utf-8')
            raiz = ET.fromstring(contenido)
            for elemento in raiz.findall('.//item', espacios_nombres):
                titulo = elemento.find('title').text if elemento.find('title') is not None else "Sin título"
                enlace = elemento.find('link').text if elemento.find('link') is not None else "Sin link"
                descripcion = elemento.find('description').text if elemento.find('description') is not None else "Sin descripción"
                fecha_str = elemento.find('pubDate').text if elemento.find('pubDate') is not None else "Sin fecha"

                # Formatear la fecha
                fecha = datetime.strptime(fecha_str, '%a, %d %b %Y %H:%M:%S %Z')
                fecha_formateada = fecha.strftime('%d/%m/%Y')

                seccion = elemento.find('category').text if elemento.find('category') is not None else "Sin Cat"

                escritor.writerow(['Expansión', titulo, descripcion, seccion, url, fecha_formateada])
        else:
            print(f"Error al acceder URSS: {respuesta.status_code} en URL {url}")

print('Datos guardados en el archivo CSV.')


Datos guardados en el archivo CSV.


**Generar CSV normalizado**

In [ ]:
#por si se reinicia
!python -m spacy download es_core_news_sm
!pip install spacy
!pip install torch
# Descargar las stopwords en español de NLTK
nltk.download('stopwords')
# Descargar el lematizador de NLTK
nltk.download('wordnet')



In [8]:
#V2 CODIGO con libreria nltk

import pandas as pd
import nltk
from nltk.corpus import stopwords

# Descargar las stopwords en español de NLTK
stop_words = set(stopwords.words('spanish'))

# Cargar el archivo CSV
df = pd.read_csv('Expansion.csv', sep=',', header=None, engine='python')

# Función para normalizar el texto
def normalize_text(text):
    tokens = text.split()
    normalized_tokens = []
    for token in tokens:
        if token.lower() not in stop_words:
            normalized_tokens.append(token)
    return ' '.join(normalized_tokens)

# Normalizar el texto de las columnas 'Título' y 'Resumen de Contenido'
df[1] = df[1].apply(normalize_text)  # Título
df[2] = df[2].apply(normalize_text)  # Resumen de Contenido

# Guardar el dataframe modificado en un nuevo archivo CSV
df.to_csv('normalized_data_corpus.csv', index=False, header=None)
print('Datos guardados en el archivo CSV.')


Datos guardados en el archivo CSV.


In [11]:
#V3 normalizado y lematizado
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
stop_words = set(stopwords.words('spanish'))
lemmatizer = WordNetLemmatizer()

# Cargar el archivo CSV
df = pd.read_csv('Expansion.csv', sep=',', header=None, engine='python')

# Función para normalizar el texto
def normalize_text(text):
    tokens = text.split()
    normalized_tokens = []
    for token in tokens:
        if token.lower() not in stop_words:
            normalized_tokens.append(lemmatizer.lemmatize(token.lower()))  # Lemmatization
    return ' '.join(normalized_tokens)

# Normalizar el texto de las columnas 'Título' y 'Resumen de Contenido'
df[1] = df[1].apply(normalize_text)  # Título
df[2] = df[2].apply(normalize_text)  # Resumen de Contenido

# Guardar el dataframe modificado en un nuevo archivo CSV
df.to_csv('normalized_data_corpus.csv', index=False, header=None)
print('Datos guardados en el archivo CSV.')


Datos guardados en el archivo CSV.
